## Introducción

En este proyecto analizaré la base de datos de una tienda online de venta de videojuegos, con actividades en varios países. 
Mi objetivo principal es identificar patrones que determinen cuales son los juegos más exitosos y cuales no, lo cual permitirá plantear nuevos proyectos y diseñar campañas publicitarias eficaces para el próximo año. 

Con mi objetivo en mente, realizaré un análisis exploratorio de datos y prepararé el conjunto para trabajar con el; plantearé algunas hipótesis enfocadas en los números arrojados por las ventas en los diferentes países y las críticas de los usuarios y la junta de entretenimiento; mediante la prueba de esas hipótesis determinaré cuales videojuegos y las plataformas son más exitosas. 

## Inicialización

In [1]:
#importar librerías
import pandas as pd
import numpy as np
from scipy import stats as st
import math as mt
from math import factorial
from matplotlib import pyplot as plt
import seaborn as sns
import plotly.express as px

## Cargar datos

In [2]:
#leer el conjunto de datos en un dataframe
games = pd.read_csv("dataset/games.csv")

## Preparar los datos

In [3]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16713 non-null  object 
 1   Platform         16715 non-null  object 
 2   Year_of_Release  16446 non-null  float64
 3   Genre            16713 non-null  object 
 4   NA_sales         16715 non-null  float64
 5   EU_sales         16715 non-null  float64
 6   JP_sales         16715 non-null  float64
 7   Other_sales      16715 non-null  float64
 8   Critic_Score     8137 non-null   float64
 9   User_Score       10014 non-null  object 
 10  Rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.4+ MB


In [18]:
#reemplazar nombres de columnas a minúsculas
new_col_names = []

for old_name in games.columns:
    name_lowered = old_name.lower()
    new_col_names.append(name_lowered)
    
games.columns = new_col_names
